In [2]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-19 12:48:13--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip’

dataset_tsmc2014.zi 100%[===================>]  24.36M  4.41MB/s    in 12s     

2024-05-19 12:48:26 (2.10 MB/s) - ‘dataset_tsmc2014.zip’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
   creating: dataset_tsmc2014/
  inflating: dataset_tsmc2014/dataset_TSMC2014_NYC.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_readme.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_TKY.txt  


In [3]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [4]:
df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
print(df)

        User ID                  Venue ID         Venue category ID  \
0           470  49bbd6c0f964a520f4531fe3  4bf58dd8d48988d127951735   
1           979  4a43c0aef964a520c6a61fe3  4bf58dd8d48988d1df941735   
2            69  4c5cc7b485a1e21e00d35711  4bf58dd8d48988d103941735   
3           395  4bc7086715a7ef3bef9878da  4bf58dd8d48988d104941735   
4            87  4cf2c5321d18a143951b5cec  4bf58dd8d48988d1cb941735   
...         ...                       ...                       ...   
227423      688  3fd66200f964a52000e71ee3  4bf58dd8d48988d1e7931735   
227424      560  4bca32ff0687ef3be789dbcc  4bf58dd8d48988d16c941735   
227425      945  50a77716e4b0b5a9492f6f56  4bf58dd8d48988d103941735   
227426      671  4514efe0f964a520e7391fe3  4bf58dd8d48988d11d941735   
227427      942  4a1e0ca0f964a520bf7b1fe3  4bf58dd8d48988d116941735   

        Venue category name   Latitude  Longitude  Timezone  \
0       Arts & Crafts Store  40.719810 -74.002581      -240   
1                    

In [ ]:
from datetime import datetime

def str2date(date_str):

    parsed_date = datetime.strptime(date_str, "%a %b %d %H:%M:%S")
    print(parsed_date)

In [ ]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [ ]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [ ]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
#print(venuesdf)


In [ ]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))

G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])



100%|██████████| 38332/38332 [45:54<00:00, 13.92it/s] 


"\nfor el1 in venuesdict:\nfor el2 in venuesdict:\ndistances.append(distance_between_coordinates(venuesdict[el1]['Latitude'],venuesdict[el1]['Longitude'],venuesdict[el2]['Latitude'],venuesdict[el2]['Longitude']))\n"

In [ ]:
import pickle

spatpG = G

pickle.dump(G,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

In [ ]:
#nx.draw(G)
print(G)

Graph with 38333 nodes and 15134650 edges


In [ ]:
#spatial graph construction

print(len(venuesdf))
import networkx as nx

spatG = nx.Graph()

d = 1

# Add nodes and edges with features
for _, row1 in venuesdf.iterrows():

    if not spatG.has_node(row1['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

    for _,row2 in venuesdf.iterrows():


        # Add nodes if not already in the graph


        if not spatG.has_node(row2['Venue ID']):
            spatG.add_node(row1['Venue ID'], category=row1['Venue category ID'])

        # Add edges with features

        if distance_between_coordinates(row1['Latitude'],row1['Longitude'],row2['Latitude'],row2['Longitude']) < d:

            spatG.add_edge(row1['Venue ID'],row2['Venue ID'])


print(spatG)




In [5]:
from queue import SimpleQueue
#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

usrloc = {}

seqG = nx.Graph()

for _, row in df.iterrows():

  if not seqG.has_node(row['Venue ID']):
    seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    usrloc[row['User ID']] = row['Venue ID']






In [ ]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Graph with 38333 nodes and 150166 edges


In [6]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-6-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [72]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

week['workdays'] = []
week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  if 1 <= day <=4:
    week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  else:
    week['weekend'].append([row['Venue ID'],row['Venue category ID']])






                       Venue ID         Venue category ID  localHour
0      3fd66200f964a52000e71ee3  4bf58dd8d48988d1e7931735    16.3125
1      3fd66200f964a52000e81ee3  4bf58dd8d48988d176941735    15.0000
2      3fd66200f964a52000f11ee3  4bf58dd8d48988d1e4931735    23.0000
3      3fd66200f964a52001e51ee3  4bf58dd8d48988d118941735    23.0000
4      3fd66200f964a52001e81ee3  4bf58dd8d48988d116941735    17.4000
...                         ...                       ...        ...
38674  51190f52e4b0f83278d5b9ba  4bf58dd8d48988d104941735    10.0000
38675  51194e17e4b0665cf3280ce3  4bf58dd8d48988d127941735    15.0000
38676  511aa583e4b03acfa105f50b  4e0e22f5a56208c4ea9a85a0    18.0000
38677  511ae28ce4b00516bc52ccae  4bf58dd8d48988d103941735    19.0000
38678  511b0bfae4b0acbffed9afbe  4bf58dd8d48988d1be941735    22.0000

[38679 rows x 3 columns]
                       Venue ID         Venue category ID  localHour  \
0      3fd66200f964a52000e71ee3  4bf58dd8d48988d1e7931735    16.3125   
1 

KeyboardInterrupt: 

In [73]:
dayG = nx.Graph()
hourG = nx.Graph()

for el in hours:
   for u in hours[el]:

    if not hourG.has_node(u[0]):
      hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        if not hourG.has_node(u[0]):
          hourG.add_node(v[0], category=v[1])

        hourG.add_edge(u[0],v[0])

print(hourG)



Graph with 38333 nodes and 48811796 edges


In [ ]:
for el in week:
   for u in week[el]:

    if not dayG.has_node(u[0]):
      dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        if not dayG.has_node(u[0]):
          dayG.add_node(v[0], category=v[1])

        dayG.add_edge(u[0],v[0])

print(dayG)
